In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier
from cfmining.action_set import ActionSet

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [29]:
giveme_df = pd.read_csv('data/student/student-mat.csv', sep=';', index_col=None).dropna()
y = 1*(giveme_df['G3']>=10)
X = giveme_df.drop(['G1', 'G2', 'G3'], axis=1)

In [30]:
X = giveme_df.drop(['G1', 'G2', 'G3'], axis=1)
X['school'] = X['school'].map({'GP': 0, 'MS': 1})
X['sex'] = X['sex'].map({'M': 0, 'F': 1})
X['address'] = X['address'].map({'R': 0, 'U': 1})
X['famsize'] = X['famsize'].map({'LE3': 0, 'GT3': 1})
X['Pstatus'] = X['Pstatus'].map({'A': 0, 'T': 1})
X['reason'] = X['reason'].map({'other':0, 'home':1, 'reputation':2, 'course':3})
X['Mjob'] = X['Mjob'].map({'other':0, 'health':1, 'services':2, 'at_home':3, 'teacher':4})
X['Fjob'] = X['Fjob'].map({'other':0, 'health':1, 'services':2, 'at_home':3, 'teacher':4})
X['guardian'] = X['guardian'].map({'other':0, 'mother':1, 'father':2})
X['schoolsup'] = X['schoolsup'].map({'no': 0, 'yes': 1})
X['famsup'] = X['famsup'].map({'no': 0, 'yes': 1})
X['paid'] = X['paid'].map({'no': 0, 'yes': 1})
X['activities'] = X['activities'].map({'no': 0, 'yes': 1})
X['nursery'] = X['nursery'].map({'no': 0, 'yes': 1})
X['higher'] = X['higher'].map({'no': 0, 'yes': 1})
X['internet'] = X['internet'].map({'no': 0, 'yes': 1})
X['romantic'] = X['romantic'].map({'no': 0, 'yes': 1})
for name in X:
    print(name, X[name].unique())

school [0 1]
sex [1 0]
age [18 17 15 16 19 22 20 21]
address [1 0]
famsize [1 0]
Pstatus [0 1]
Medu [4 1 3 2 0]
Fedu [4 1 2 3 0]
Mjob [3 1 0 2 4]
Fjob [4 0 2 1 3]
reason [3 0 1 2]
guardian [1 2 0]
traveltime [2 1 3 4]
studytime [2 3 1 4]
failures [0 3 2 1]
schoolsup [1 0]
famsup [0 1]
paid [0 1]
activities [0 1]
nursery [1 0]
higher [1 0]
internet [0 1]
romantic [0 1]
famrel [4 5 3 1 2]
freetime [3 2 4 1 5]
goout [4 3 2 1 5]
Dalc [1 2 5 3 4]
Walc [1 3 2 4 5]
health [3 5 1 2 4]
absences [ 6  4 10  2  0 16 14  7  8 25 12 54 18 26 20 56 24 28  5 13 15 22  3 21
  1 75 30 19  9 11 38 40 23 17]


In [31]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

## Setting ActionSet base parameters

In [66]:
pd.Series(clf_lgr['clf'].coef_[0], index=X.columns).sort_values()

failures     -0.121202
goout        -0.078400
age          -0.073083
famsize      -0.058906
sex          -0.049124
absences     -0.042848
Pstatus      -0.041503
famsup       -0.040179
reason       -0.039838
romantic     -0.035262
Dalc         -0.027709
activities   -0.025095
health       -0.023446
schoolsup    -0.020856
freetime     -0.008175
nursery      -0.005179
Walc         -0.004745
Mjob         -0.002630
traveltime   -0.002461
school       -0.001169
guardian      0.001858
internet      0.007202
Fjob          0.007780
studytime     0.017566
Medu          0.024317
address       0.031727
paid          0.031842
Fedu          0.035208
famrel        0.038337
higher        0.058774
dtype: float64

In [101]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['sex', 'age']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = -1
    if feat.name in ['Mjob', 'Fjob']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = 1
    if feat.name in ['famsize', 'traveltime', 'failures',
                    'romantic', 'Dalc', 'Walc', 'absences',
                     'goout', 'Pstatus', 'reason', 'activities',
                    'schoolsup',]:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = -1
    if feat.name in ['address', 'Medu',
                     'Fedu', 'guardian',
                    'studytime', 'schoolsup', 'famsup',
                     'paid', 'nursery',
                     'higher', 'internet', 'famrel',
                    'freetime', 'health']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = 1
    feat.step_type = 'absolute'
    if feat.name in ['absences']:
        feat.step_type = 'relative'
        feat.step_size = 0.1
    feat.step_size = 1
    print(feat.name, feat.step_direction, feat.step_type, feat.step_size)
    print(X[feat.name].unique())
    feat.update_grid()

school 0 absolute 1.0
[0 1]
sex -1 absolute 1.0
[1 0]
age -1 absolute 1.0
[18 17 15 16 19 22 20 21]
address 1 absolute 1.0
[1 0]
famsize -1 absolute 1.0
[1 0]
Pstatus -1 absolute 1.0
[0 1]
Medu 1 absolute 1.0
[4 1 3 2 0]
Fedu 1 absolute 1.0
[4 1 2 3 0]
Mjob 1 absolute 1.0
[3 1 0 2 4]
Fjob 1 absolute 1.0
[4 0 2 1 3]
reason -1 absolute 1.0
[3 0 1 2]
guardian 1 absolute 1.0
[1 2 0]
traveltime -1 absolute 1.0
[2 1 3 4]
studytime 1 absolute 1.0
[2 3 1 4]
failures -1 absolute 1.0
[0 3 2 1]
schoolsup 1 absolute 1.0
[1 0]
famsup 1 absolute 1.0
[0 1]
paid 1 absolute 1.0
[0 1]
activities -1 absolute 1.0
[0 1]
nursery 1 absolute 1.0
[1 0]
higher 1 absolute 1.0
[1 0]
internet 1 absolute 1.0
[0 1]
romantic -1 absolute 1.0
[0 1]
famrel 1 absolute 1.0
[4 5 3 1 2]
freetime 1 absolute 1.0
[3 2 4 1 5]
goout -1 absolute 1.0
[4 3 2 1 5]
Dalc -1 absolute 1.0
[1 2 5 3 4]
Walc -1 absolute 1.0
[1 3 2 4 5]
health 1 absolute 1.0
[3 5 1 2 4]
absences -1 relative 1.0
[ 6  4 10  2  0 16 14  7  8 25 12 54 18 26 20 

## Logistic regression model

In [102]:
y

0      0
1      0
2      1
3      1
4      1
      ..
390    0
391    1
392    0
393    1
394    0
Name: G3, Length: 395, dtype: int64

In [103]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [104]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=0.5)
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_

## Preparing counterfactual parameters

In [105]:
action_set = copy.deepcopy(action_set_base)

print('ActionSet stats')
print('Number of actionable features:', sum([action.actionable for action in action_set]))
print('Mean number of actions per feature:', np.nanmean([len(action._grid) if action.actionable else np.nan for action in action_set]))
print('Max number of actions per feature:', np.nanmax([len(action._grid) if action.actionable else np.nan for action in action_set]))

ActionSet stats
Number of actionable features: 26
Mean number of actions per feature: 3.1923076923076925
Max number of actions per feature: 5.0


In [106]:
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .5].index

In [107]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [108]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, 0.5)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = [int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)]
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if ~any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

100%|██████████| 50/50 [00:01<00:00, 32.15it/s]


## Greedy approach

In [109]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 50/50 [00:02<00:00, 22.55it/s]


## MAPOCAM

In [110]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 50/50 [00:10<00:00,  4.60it/s]


## Performance comparison for single objective


In [111]:
mapocam_lgr_costs

array([0.06837596, 0.29270725, 0.1266171 , 0.1266171 , 0.21855093,
       0.17242426, 0.36581373, 0.13388172, 0.06595357, 0.1302299 ,
       0.21855093, 0.13388172, 0.19040458, 0.21855093, 0.36581373,
       0.13388172, 0.1266171 , 0.32772814, 0.1266171 , 0.32772814,
       0.16468274, 0.13388172, 0.32772814, 0.21855093, 0.13388172,
       0.12911386, 0.13388172, 0.4582275 , 0.16966099, 0.32772814,
       0.1266171 , 0.07156859, 0.2267331 , 0.24305853, 0.21855093,
       0.21855093, 0.13388172, 0.21855093, 0.26681248, 0.32772814,
       0.26839241, 0.12911386, 0.29270725, 0.35243265, 0.16966099,
       0.33154346, 0.25473642, 0.02284535, 0.36581373, 0.33417698])

In [112]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

In [113]:
print('Time performance')
print('Ustun:', np.mean(lp_lgr_time))
print('Greedy:', np.mean(greedy_lgr_time))
print('Proposed:', np.mean(mapocam_lgr_time))

Time performance
Ustun: 0.03008054511999944
Greedy: 0.04386185822062544
Proposed: 0.2166017066994391


In [114]:
print('Best solution rate')
best_costs = np.min([lp_lgr_costs, greedy_lgr_costs, mapocam_lgr_costs], axis=0)
print('Ustun:', np.mean(lp_lgr_costs==best_costs))
print('Greedy:', np.mean(greedy_lgr_costs==best_costs))
print('Proposed:', np.mean(mapocam_lgr_costs==best_costs))

Best solution rate
Ustun: 1.0
Greedy: 0.38
Proposed: 1.0


## Comparing Pareto enumeration

## Percentile vs changes

In [115]:
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 50/50 [00:48<00:00,  1.03it/s]


In [116]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    0.8, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 50/50 [00:00<00:00, 73.33it/s]


In [117]:
bf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_pc[i] = {}
    bf_results_pc[i]['enum'] = bf
    bf_results_pc[i]['time'] = bf_time
    bf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 50/50 [1:19:31<00:00, 95.43s/it] 


## Features

In [118]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 50/50 [00:33<00:00,  1.47it/s]


In [119]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    0.8, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 50/50 [00:00<00:00, 73.20it/s]


In [120]:
bf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_feat[i] = {}
    bf_results_feat[i]['enum'] = bf
    bf_results_feat[i]['time'] = bf_time
    bf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 50/50 [07:56<00:00,  9.53s/it]


In [122]:
brute_sols_pc = np.array([bf_results_pc[i]['num'] for i in denied_individuals])
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])

brute_sols_feat = np.array([bf_results_feat[i]['num'] for i in denied_individuals])
mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])

brute_time_pc = np.array([bf_results_pc[i]['time'] for i in denied_individuals])
mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

brute_time_feat = np.array([bf_results_feat[i]['time'] for i in denied_individuals])
mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

## Testing same number of solutions


In [123]:
print('Changes vs percentile:', np.mean(brute_sols_pc==mapocam_sols_pc)==1, np.mean(lp_sols_pc==mapocam_sols_pc)==1)
print('Features:', np.mean(brute_sols_feat==mapocam_sols_feat)==1, np.mean(lp_sols_feat==mapocam_sols_feat)==1)

Changes vs percentile: True False
Features: True False


In [124]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_time_pc))
print('Brute force:',np.mean(brute_time_pc))
print('Proposed:', np.mean(mapocam_time_pc))
print('Features:')
print('Ustun:',np.mean(lp_time_feat))
print('Brute force:',np.mean(brute_time_feat))
print('Proposed:', np.mean(mapocam_time_feat))

Changes vs percentile:
Ustun: 0.013255503939872142
Brute force: 95.42343208616032
Proposed: 0.9685787341797549
Features:
Ustun: 0.013348832940246212
Brute force: 9.530755202900181
Proposed: 0.679174274179677


## Random forest

In [ ]:
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

In [ ]:
clf_rt = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5, max_leaf_nodes=31, class_weight='balanced_subsample')
clf_rt.fit(Xtr, ytr);

## Preparing counterfactual parameters

In [ ]:
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction
    
print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_tree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_tree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_tree]))

In [ ]:
scores = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .8].index

In [ ]:
percCalc = PercentileCalculator(action_set=action_set_tree)

In [ ]:
mapocam_clf_rf = TreeClassifier(clf_rt, Xtr, ytr, threshold=0.8)
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=0.8, use_predict_max=True)

## Linear programming

In [ ]:
lp_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=0.8, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_rf_results[i] = {}
    lp_rf_results[i]['enum'] = lp
    lp_rf_results[i]['indiv'] = individual
    lp_rf_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_rf_results[i]['time'] = lp_time
    lp_rf_results[i]['cost'] = criteria.f(lp_rf_results[i]['solution']) if not any(np.isnan(lp_rf_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    greedy = Greedy(action_set_tree, individual, mapocam_clf_rf_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_rf_results[i] = {}
    greedy_rf_results[i]['solution'] = greedy.solution
    greedy_rf_results[i]['time'] = greedy_time
    greedy_rf_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [ ]:
mapocam_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results[i] = {}
    mapocam_rf_results[i]['enum'] = mapocam
    mapocam_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results[i]['time'] = mapocam_time
    mapocam_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_rf_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_plus[i] = {}
    mapocam_rf_results_plus[i]['enum'] = mapocam
    mapocam_rf_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results_plus[i]['time'] = mapocam_time
    mapocam_rf_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
lp_rf_costs = np.array([lp_rf_results[i]['cost'] for i in denied_individuals])
greedy_rf_costs = np.array([greedy_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs = np.array([mapocam_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs_plus = np.array([mapocam_rf_results_plus[i]['cost'] for i in denied_individuals])

lp_rf_time = np.array([lp_rf_results[i]['time'] for i in denied_individuals])
greedy_rf_time = np.array([greedy_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time = np.array([mapocam_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time_plus = np.array([mapocam_rf_results_plus[i]['time'] for i in denied_individuals])

In [ ]:
print('Time performance')
print('Ustun:', np.mean(lp_rf_time))
print('Greedy:', np.mean(greedy_rf_time))
print('Proposed:', np.mean(mapocam_rf_time))
print('Proposed plus:', np.mean(mapocam_rf_time_plus))

In [ ]:
print('Best solution rate')
best_costs = np.min([lp_rf_costs, greedy_rf_costs, mapocam_rf_costs, mapocam_rf_costs_plus], axis=0)
print('Ustun:', np.mean(lp_rf_costs==best_costs))
print('Greedy:', np.mean(greedy_rf_costs==best_costs))
print('Proposed:', np.mean(mapocam_rf_costs==best_costs))
print('Proposed plus:', np.mean(mapocam_rf_costs_plus==best_costs))

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc[i] = {}
    mapocam_rf_results_pc[i]['enum'] = mapocam
    mapocam_rf_results_pc[i]['time'] = mapocam_time
    mapocam_rf_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_rf_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc_plus[i] = {}
    mapocam_rf_results_pc_plus[i]['enum'] = mapocam
    mapocam_rf_results_pc_plus[i]['time'] = mapocam_time
    mapocam_rf_results_pc_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=0.6,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_pc[i] = {}
    lp_rf_results_pc[i]['enum'] = lp
    lp_rf_results_pc[i]['time'] = lp_time
    lp_rf_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_pc[i] = {}
    bf_rf_results_pc[i]['enum'] = bf
    bf_rf_results_pc[i]['time'] = bf_time
    bf_rf_results_pc[i]['num'] = len(bf.solutions)

## Features

In [ ]:
mapocam_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat[i] = {}
    mapocam_rf_results_feat[i]['enum'] = mapocam
    mapocam_rf_results_feat[i]['time'] = mapocam_time
    mapocam_rf_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=0.8, use_predict_max=True)

In [ ]:
mapocam_rf_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat_plus[i] = {}
    mapocam_rf_results_feat_plus[i]['enum'] = mapocam
    mapocam_rf_results_feat_plus[i]['time'] = mapocam_time
    mapocam_rf_results_feat_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])
print('Proposed:', np.mean(mapocam_rf_time_feat_plus))

In [ ]:
lp_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=0.8,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_feat[i] = {}
    lp_rf_results_feat[i]['enum'] = lp
    lp_rf_results_feat[i]['time'] = lp_time
    lp_rf_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_feat[i] = {}
    bf_rf_results_feat[i]['enum'] = bf
    bf_rf_results_feat[i]['time'] = bf_time
    bf_rf_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_rf_sols_pc = np.array([bf_rf_results_pc[i]['num'] for i in denied_individuals])
lp_rf_sols_pc = np.array([lp_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc = np.array([mapocam_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['num'] for i in denied_individuals])

brute_rf_sols_feat = np.array([bf_rf_results_feat[i]['num'] for i in denied_individuals])
lp_rf_sols_feat = np.array([lp_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat = np.array([mapocam_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['num'] for i in denied_individuals])

brute_rf_time_pc = np.array([bf_rf_results_pc[i]['time'] for i in denied_individuals])
lp_rf_time_pc = np.array([lp_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc = np.array([mapocam_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['time'] for i in denied_individuals])

brute_rf_time_feat = np.array([bf_rf_results_feat[i]['time'] for i in denied_individuals])
lp_rf_time_feat = np.array([lp_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat = np.array([mapocam_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])

In [ ]:
print('Changes vs percentile:', np.mean(brute_rf_sols_pc==mapocam_rf_sols_pc)==1, np.mean(lp_rf_sols_pc==mapocam_rf_sols_pc)==1, np.mean(mapocam_rf_sols_pc_plus==mapocam_rf_sols_pc)==1)
print('Features:', np.mean(brute_rf_sols_feat==mapocam_rf_sols_feat)==1, np.mean(lp_rf_sols_feat==mapocam_rf_sols_feat)==1, np.mean(mapocam_rf_sols_feat_plus==mapocam_rf_sols_feat)==1)

In [ ]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_rf_time_pc))
print('Brute force:',np.mean(brute_rf_time_pc))
print('Proposed:', np.mean(mapocam_rf_time_pc))
print('Proposed plus:', np.mean(mapocam_rf_time_pc_plus))
print('Features:')
print('Ustun:',np.mean(lp_rf_time_feat))
print('Brute force:',np.mean(brute_rf_time_feat))
print('Proposed:', np.mean(mapocam_rf_time_feat))
print('Proposed plus:', np.mean(mapocam_rf_time_feat_plus))

## Monotone trees

In [ ]:
from sklearn import tree, ensemble
import lightgbm
from cfmining.predictors import MonotoneTree

In [ ]:
monotone_constraints = [action.step_direction for action in action_set_base]
clf_mt = lightgbm.LGBMClassifier(n_estimators=10, max_depth=10, num_leaves=63, monotone_constraints=monotone_constraints)#, class_weight='balanced')
clf_mt.fit(Xtr, ytr)

## Preparing counterfactual parameters

In [ ]:
action_set_mtree = copy.deepcopy(action_set_base)
action_set_mtree.embed_forest(clf_mt)
for action in action_set_mtree:
    action.flip_direction = action.step_direction

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_mtree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_mtree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_mtree]))

In [ ]:
scores = pd.Series(clf_mt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .8].index

In [ ]:
percCalc = PercentileCalculator(action_set=action_set_mtree)

In [ ]:
mapocam_clf_mt = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=0.8)
mapocam_clf_mt_fast = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=0.8, use_predict_max=True)
mapocam_clf_mt_mon = MonotoneTree(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=0.8, use_predict_max=True)

## Linear programming

In [ ]:
lp_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=0.8, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_mt_results[i] = {}
    lp_mt_results[i]['enum'] = lp
    lp_mt_results[i]['indiv'] = individual
    lp_mt_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_mt_results[i]['time'] = lp_time
    lp_mt_results[i]['cost'] = criteria.f(lp_mt_results[i]['solution']) if not any(np.isnan(lp_mt_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    greedy = Greedy(action_set_mtree, individual, mapocam_clf_mt, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_mt_results[i] = {}
    greedy_mt_results[i]['solution'] = greedy.solution
    greedy_mt_results[i]['time'] = greedy_time
    greedy_mt_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAME

In [ ]:
mapocam_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results[i] = {}
    mapocam_mt_results[i]['enum'] = mapocam
    mapocam_mt_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results[i]['time'] = mapocam_time
    mapocam_mt_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_mt_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_plus[i] = {}
    mapocam_mt_results_plus[i]['enum'] = mapocam
    mapocam_mt_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_plus[i]['time'] = mapocam_time
    mapocam_mt_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_mt_results_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_mon[i] = {}
    mapocam_mt_results_mon[i]['enum'] = mapocam
    mapocam_mt_results_mon[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_mon[i]['time'] = mapocam_time
    mapocam_mt_results_mon[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
lp_mt_costs = np.array([lp_mt_results[i]['cost'] for i in denied_individuals])
greedy_mt_costs = np.array([greedy_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs = np.array([mapocam_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_plus = np.array([mapocam_mt_results_plus[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_mon = np.array([mapocam_mt_results_mon[i]['cost'] for i in denied_individuals])

lp_mt_time = np.array([lp_mt_results[i]['time'] for i in denied_individuals])
greedy_mt_time = np.array([greedy_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time = np.array([mapocam_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time_plus = np.array([mapocam_mt_results_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_mon = np.array([mapocam_mt_results_mon[i]['time'] for i in denied_individuals])

In [ ]:
print('Time performance')
print('Ustun:', np.mean(lp_mt_time))
print('Greedy:', np.mean(greedy_mt_time))
print('Proposed:', np.mean(mapocam_mt_time))
print('Proposed plus:', np.mean(mapocam_mt_time_plus))
print('Proposed mon:', np.mean(mapocam_mt_time_mon))

In [ ]:
print('Best solution rate')
#best_costs = np.min([lp_mt_costs, greedy_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus], axis=0)
best_costs = np.min([lp_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus], axis=0)
print('Ustun:', np.mean(lp_mt_costs==best_costs))
print('Greedy:', np.mean(greedy_mt_costs==best_costs))
print('Proposed:', np.mean(mapocam_mt_costs==best_costs))
print('Proposed plus:', np.mean(mapocam_mt_costs_plus==best_costs))
print('Proposed mon:', np.mean(mapocam_mt_costs_mon==best_costs))

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc[i] = {}
    mapocam_mt_results_pc[i]['enum'] = mapocam
    mapocam_mt_results_pc[i]['time'] = mapocam_time
    mapocam_mt_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_plus[i] = {}
    mapocam_mt_results_pc_plus[i]['enum'] = mapocam
    mapocam_mt_results_pc_plus[i]['time'] = mapocam_time
    mapocam_mt_results_pc_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_pc_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_mon[i] = {}
    mapocam_mt_results_pc_mon[i]['enum'] = mapocam
    mapocam_mt_results_pc_mon[i]['time'] = mapocam_time
    mapocam_mt_results_pc_mon[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=0.8,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_pc[i] = {}
    lp_mt_results_pc[i]['enum'] = lp
    lp_mt_results_pc[i]['time'] = lp_time
    lp_mt_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_pc[i] = {}
    bf_mt_results_pc[i]['enum'] = bf
    bf_mt_results_pc[i]['time'] = bf_time
    bf_mt_results_pc[i]['num'] = len(bf.solutions)

## Features

In [ ]:
mapocam_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat[i] = {}
    mapocam_mt_results_feat[i]['enum'] = mapocam
    mapocam_mt_results_feat[i]['time'] = mapocam_time
    mapocam_mt_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_plus[i] = {}
    mapocam_mt_results_feat_plus[i]['enum'] = mapocam
    mapocam_mt_results_feat_plus[i]['time'] = mapocam_time
    mapocam_mt_results_feat_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_feat_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_mon[i] = {}
    mapocam_mt_results_feat_mon[i]['enum'] = mapocam
    mapocam_mt_results_feat_mon[i]['time'] = mapocam_time
    mapocam_mt_results_feat_mon[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=0.8,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_feat[i] = {}
    lp_mt_results_feat[i]['enum'] = lp
    lp_mt_results_feat[i]['time'] = lp_time
    lp_mt_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_feat[i] = {}
    bf_mt_results_feat[i]['enum'] = bf
    bf_mt_results_feat[i]['time'] = bf_time
    bf_mt_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_mt_sols_pc = np.array([bf_mt_results_pc[i]['num'] for i in denied_individuals])
lp_mt_sols_pc = np.array([lp_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc = np.array([mapocam_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['num'] for i in denied_individuals])

brute_mt_sols_feat = np.array([bf_mt_results_feat[i]['num'] for i in denied_individuals])
lp_mt_sols_feat = np.array([lp_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat = np.array([mapocam_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['num'] for i in denied_individuals])

brute_mt_time_pc = np.array([bf_mt_results_pc[i]['time'] for i in denied_individuals])
lp_mt_time_pc = np.array([lp_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc = np.array([mapocam_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['time'] for i in denied_individuals])

brute_mt_time_feat = np.array([bf_mt_results_feat[i]['time'] for i in denied_individuals])
lp_mt_time_feat = np.array([lp_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat = np.array([mapocam_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['time'] for i in denied_individuals])

In [ ]:
print('Changes vs percentile:', np.mean(brute_mt_sols_pc==mapocam_mt_sols_pc)==1, np.mean(lp_mt_sols_pc==mapocam_mt_sols_pc)==1, np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc)==1)
print('Features:', np.mean(brute_mt_sols_feat==mapocam_mt_sols_feat)==1, np.mean(lp_mt_sols_feat==mapocam_mt_sols_feat)==1, np.mean(mapocam_mt_sols_feat_plus==mapocam_mt_sols_feat)==1)

In [ ]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_mt_time_pc))
print('Brute force:',np.mean(brute_mt_time_pc))
print('Proposed:', np.mean(mapocam_mt_time_pc))
print('Proposed plus:', np.mean(mapocam_mt_time_pc_plus))
print('Proposed mon:', np.mean(mapocam_mt_time_pc_mon))
print('Features:')
print('Ustun:',np.mean(lp_mt_time_feat))
print('Brute force:',np.mean(brute_mt_time_feat))
print('Proposed:', np.mean(mapocam_mt_time_feat))
print('Proposed plus:', np.mean(mapocam_mt_time_feat_plus))
print('Proposed mon:', np.mean(mapocam_mt_time_feat_mon))